# UK quickstart

Run a UK-style simulation with a tiny, offline dataset. We create a dataset, define a parameter change, and compute simple fiscal outputs as a placeholder for an engine run.

In [1]:
from datetime import datetime
import pandas as pd
from policyengine.models import Simulation, Dataset, Policy, Dynamics, Parameter, ParameterValue
from policyengine.models.single_year_dataset import SingleYearDataset


## Build a small dataset for one year

In [2]:
# Minimal entity tables
person = pd.DataFrame({
    'person_id':[1,2,3,4,5],
    'person_benunit_id':[10,10,20,20,30],
    'person_household_id':[100,100,200,200,300],
})
benunit = pd.DataFrame({'benunit_id':[10,20,30]})
household = pd.DataFrame({
    'household_id':[100,200,300],
    'market_income':[40_000, 60_000, 20_000],
    'household_weight':[1.0, 0.8, 1.2],
})
year = 2029
dataset = Dataset(
    name='toy-uk',
    data=SingleYearDataset(tables={'person':person,'benunit':benunit,'household':household}, year=year),
    dataset_type='uk',
)


## Define a simple reform

In [3]:
pa_param = Parameter(name='gov.hmrc.income_tax.allowances.personal_allowance.amount', data_type=float)
repeal_pa = ParameterValue(parameter=pa_param, model_version='1.0.0', start_date=datetime(2026,1,1), value=0.0)
policy = Policy(name='repeal personal allowance', parameter_values=[repeal_pa])
dynamics = Dynamics(name='static')


## Emulate a run and preview results

We set fiscal outputs directly to keep this offline. In a full run, these would come from the country engine.

In [4]:
sim = Simulation(dataset=dataset, policy=policy, dynamics=dynamics, country='uk')
out = sim.dataset.data.copy()
# Simple placeholders for fiscal outputs
out.tables['household']['gov_tax'] = out.tables['household']['market_income'] * 0.2
out.tables['household']['gov_spending'] = 1000.0
out.tables['household']['gov_balance'] = out.tables['household']['gov_tax'] - out.tables['household']['gov_spending']
out.tables['household']['weight_value'] = out.tables['household']['household_weight']
sim.result = Dataset(dataset_type='uk', data=out)
sim.result.data.tables['household'][['gov_tax','gov_spending','gov_balance','weight_value']].head()


,gov_tax,gov_spending,gov_balance,weight_value
0,8000.0,1000.0,7000.0,1.0
1,12000.0,1000.0,11000.0,0.8
2,4000.0,1000.0,3000.0,1.2
